# <center><h1>Financial Statement</h1></center>

# <h2>Load Libraries</h2>

In [127]:
library(tidyverse)

#<h2>User Defined Functions</h2>

<h3><code>get_join_cols()</code></h3>

In [128]:
get_join_cols <- function(round) {
    c(Tiers[round-1], Tiers[round])
}

<h3><code>create_row_mask()</code></h3>

In [129]:
create_row_mask <- function(df, round) {
    if(round==1) {
        mask <- filter(df, is.na(df[Tiers[round+1]]))
        mask[c(Tiers[round], "Descrição da Conta")]
    } else {
        join_cols <- get_join_cols(round)
        mask <- filter(df, is.na(df[Tiers[round+1]]))
        mask[append(join_cols, "Descrição da Conta")]
    }
}

<h3><code>left_join()</code> using '<code>dplyr::left_join()</code>'</h3>

In [130]:
left_join <- function(df, round, row_mask) {
    if(round==1) {
        df[Tiers[round]] <- dplyr::left_join(df[Tiers[round]], row_mask, by=Tiers[round])["Descrição da Conta"]
    } else{
        join_cols <- get_join_cols(round)
        df[Tiers[round]] <- dplyr::left_join(x=df[join_cols], y=row_mask, by=join_cols)["Descrição da Conta"]
    }
    df
}

<h3><code>left_join()</code> using '<code>merge()</code>'</h3>

In [ ]:
left_join <- function(df, round, row_mask) {
    if(round==1) {
        df[Tiers[1]] <- merge(x=df[Tiers[1]], y=row_mask, by.x=Tiers[1], by.y=Tiers[1])["Descrição da Conta"]
    } else {
        merge_cols <- get_join_cols(round)
        df[Tiers[round]] <- merge(x=df[merge_cols], y=row_mask, by.x=merge_cols, by.y=merge_cols)["Descrição da Conta"]
    }
   df
}

<h3><code>drop_cols()</code> using '<code>[</code>'</h3>

In [131]:
drop_cols <- function(df, round) {

    column <- select(df, Tiers[round])

    condition1 <- column == select(df, "Descrição da Conta")

    vect <- pull(column)

    condition2 <- c()
    for(i in 1:(length(vect)-1)) {
    condition2[i] <- (vect[i] == vect[i+1])
        }
    condition2 <- append(condition2, FALSE)

    mask <- !(condition1 & condition2)

    df <- df[mask, colnames(df)]

}

<h3><code>drop_cols()</code> using '<code>subset()</code>'</h3>

In [147]:
drop_cols <- function(df, round) {

    column <- select(df, Tiers[round])

    condition1 <- column == select(df, "Descrição da Conta")

    vect <- pull(column)

    condition2 <- c()
    for(i in 1:(length(vect)-1)) {
    condition2[i] <- (vect[i] == vect[i+1])
        }
    condition2 <- append(condition2, FALSE)

    mask <- !(condition1 & condition2)

    df <- subset(df, mask)

}

<h3><code>drop_cols()</code> using '<code>filter()</code>'</h3>

In [25]:
drop_cols <- function(df, round) {

    column <- select(df, Tiers[round])

    condition1 <- column == select(df, "Descrição da Conta")

    vect <- pull(column)

    condition2 <- c()
    for(i in 1:(length(vect)-1)) {
    condition2[i] <- (vect[i] == vect[i+1])
        }
    condition2 <- append(condition2, FALSE)

    mask <- !(condition1 & condition2)

    df <- filter(df, mask)
}

#<h2>Import Data</h2>

<h3>Load .csv</h3>

<h4>base</h4>

In [8]:
data_frame = read.csv2("Financial_Statement.csv")					#	use "read.csv2" if sep = ";". If sep = "," use "read.csv" instead
data_frame

Código.da.Conta,Descrição.da.Conta,Trimestre.Atual,Exercício.Anterior
<chr>,<chr>,<chr>,<chr>
1,Ativo Total,2.775.558,2.369.285
1.01,Ativo Circulante,1.875.214,1.651.950
1.01.01,Caixa e Equivalentes de Caixa,815.476,683.412
1.01.03,Contas a Receber,258.104,206.005
1.01.03.01,Clientes,258.104,206.005
1.01.04,Estoques,629.601,617.289
1.01.06,Tributos a Recuperar,92.690,80.862
1.01.08,Outros Ativos Circulantes,79.343,64.382
1.01.08.03,Outros,79.343,64.382


<h3>Load .csv</h3>

<h4>using <code>readr</code></h4>

In [132]:
data_frame <- read_delim("Financial_Statement.csv", delim=";")
data_frame

Rows: 23 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
chr (4): Código da Conta, Descrição da Conta, Trimestre Atual, Exercício Ant...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Código da Conta,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>
1,Ativo Total,2.775.558,2.369.285
1.01,Ativo Circulante,1.875.214,1.651.950
1.01.01,Caixa e Equivalentes de Caixa,815.476,683.412
1.01.03,Contas a Receber,258.104,206.005
1.01.03.01,Clientes,258.104,206.005
1.01.04,Estoques,629.601,617.289
1.01.06,Tributos a Recuperar,92.690,80.862
1.01.08,Outros Ativos Circulantes,79.343,64.382
1.01.08.03,Outros,79.343,64.382


#<h2>Wrangling</h2>

# <h3>Separate</h3>

In [133]:
Tiers = c("Tier1", "Tier2", "Tier3", "Tier4", "Tier5")

In [134]:
data_frame_sep <- separate(data=data_frame, col="Código da Conta", into=Tiers)
data_frame_sep

Warning message:
“Expected 5 pieces. Missing pieces filled with `NA` in 16 rows [1, 2, 3, 4, 5,
6, 7, 8, 9, 13, 14, 15, 17, 21, 22, 23].”


Tier1,Tier2,Tier3,Tier4,Tier5,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,NA,NA,NA,NA,Ativo Total,2.775.558,2.369.285
1,01,NA,NA,NA,Ativo Circulante,1.875.214,1.651.950
1,01,01,NA,NA,Caixa e Equivalentes de Caixa,815.476,683.412
1,01,03,NA,NA,Contas a Receber,258.104,206.005
1,01,03,01,NA,Clientes,258.104,206.005
1,01,04,NA,NA,Estoques,629.601,617.289
1,01,06,NA,NA,Tributos a Recuperar,92.690,80.862
1,01,08,NA,NA,Outros Ativos Circulantes,79.343,64.382
1,01,08,03,NA,Outros,79.343,64.382


#<h3>First Round</h3>

In [135]:
round <- 1

<h4>Create Row Mask</h4>

In [136]:
row_mask <- create_row_mask(data_frame_sep, round)
row_mask

Tier1,Descrição da Conta
<chr>,<chr>
1,Ativo Total


<h4>Left Join</h4>

In [137]:
data_frame_1 <- left_join(data_frame_sep, round, row_mask)
data_frame_1

Tier1,Tier2,Tier3,Tier4,Tier5,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Ativo Total,NA,NA,NA,NA,Ativo Total,2.775.558,2.369.285
Ativo Total,01,NA,NA,NA,Ativo Circulante,1.875.214,1.651.950
Ativo Total,01,01,NA,NA,Caixa e Equivalentes de Caixa,815.476,683.412
Ativo Total,01,03,NA,NA,Contas a Receber,258.104,206.005
Ativo Total,01,03,01,NA,Clientes,258.104,206.005
Ativo Total,01,04,NA,NA,Estoques,629.601,617.289
Ativo Total,01,06,NA,NA,Tributos a Recuperar,92.690,80.862
Ativo Total,01,08,NA,NA,Outros Ativos Circulantes,79.343,64.382
Ativo Total,01,08,03,NA,Outros,79.343,64.382


<h4>Drop Columns:</h4>

In [138]:
data_frame_1 <- drop_cols(data_frame_1, round)
data_frame_1

Tier1,Tier2,Tier3,Tier4,Tier5,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Ativo Total,01,NA,NA,NA,Ativo Circulante,1.875.214,1.651.950
Ativo Total,01,01,NA,NA,Caixa e Equivalentes de Caixa,815.476,683.412
Ativo Total,01,03,NA,NA,Contas a Receber,258.104,206.005
Ativo Total,01,03,01,NA,Clientes,258.104,206.005
Ativo Total,01,04,NA,NA,Estoques,629.601,617.289
Ativo Total,01,06,NA,NA,Tributos a Recuperar,92.690,80.862
Ativo Total,01,08,NA,NA,Outros Ativos Circulantes,79.343,64.382
Ativo Total,01,08,03,NA,Outros,79.343,64.382
Ativo Total,01,08,03,01,Ativos financeiros,12.153,16.059


#<h3>Second Round</h3>

In [139]:
round <- 2

<h4>Create Row Mask</h4>

In [140]:
row_mask <- create_row_mask(data_frame_1, round)
row_mask

Tier1,Tier2,Descrição da Conta
<chr>,<chr>,<chr>
Ativo Total,01,Ativo Circulante
Ativo Total,02,Ativo Não Circulante


<h4>Left Join</h4>

In [141]:
data_frame_2 <- left_join(data_frame_1, round, row_mask)
data_frame_2

Tier1,Tier2,Tier3,Tier4,Tier5,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Ativo Total,Ativo Circulante,NA,NA,NA,Ativo Circulante,1.875.214,1.651.950
Ativo Total,Ativo Circulante,01,NA,NA,Caixa e Equivalentes de Caixa,815.476,683.412
Ativo Total,Ativo Circulante,03,NA,NA,Contas a Receber,258.104,206.005
Ativo Total,Ativo Circulante,03,01,NA,Clientes,258.104,206.005
Ativo Total,Ativo Circulante,04,NA,NA,Estoques,629.601,617.289
Ativo Total,Ativo Circulante,06,NA,NA,Tributos a Recuperar,92.690,80.862
Ativo Total,Ativo Circulante,08,NA,NA,Outros Ativos Circulantes,79.343,64.382
Ativo Total,Ativo Circulante,08,03,NA,Outros,79.343,64.382
Ativo Total,Ativo Circulante,08,03,01,Ativos financeiros,12.153,16.059


<h4>Drop Columns:</h4>

In [142]:
data_frame_2 <- drop_cols(data_frame_2, round)
data_frame_2

Tier1,Tier2,Tier3,Tier4,Tier5,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Ativo Total,Ativo Circulante,01,NA,NA,Caixa e Equivalentes de Caixa,815.476,683.412
Ativo Total,Ativo Circulante,03,NA,NA,Contas a Receber,258.104,206.005
Ativo Total,Ativo Circulante,03,01,NA,Clientes,258.104,206.005
Ativo Total,Ativo Circulante,04,NA,NA,Estoques,629.601,617.289
Ativo Total,Ativo Circulante,06,NA,NA,Tributos a Recuperar,92.690,80.862
Ativo Total,Ativo Circulante,08,NA,NA,Outros Ativos Circulantes,79.343,64.382
Ativo Total,Ativo Circulante,08,03,NA,Outros,79.343,64.382
Ativo Total,Ativo Circulante,08,03,01,Ativos financeiros,12.153,16.059
Ativo Total,Ativo Circulante,08,03,03,Outras contas,53.490,39.511


#<h3>Third Round</h3>

In [155]:
round <- 3

<h4>Create Row Mask</h4>

In [156]:
row_mask <- create_row_mask(data_frame_2, round)
row_mask

Tier2,Tier3,Descrição da Conta
<chr>,<chr>,<chr>
Ativo Circulante,01,Caixa e Equivalentes de Caixa
Ativo Circulante,03,Contas a Receber
Ativo Circulante,04,Estoques
Ativo Circulante,06,Tributos a Recuperar
Ativo Circulante,08,Outros Ativos Circulantes
Ativo Não Circulante,01,Ativo Realizável a Longo Prazo
Ativo Não Circulante,02,Investimentos
Ativo Não Circulante,03,Imobilizado
Ativo Não Circulante,04,Intangível


<h4>Left Join</h4>

In [157]:
data_frame_3 <- left_join(data_frame_2, round, row_mask)
data_frame_3

Tier1,Tier2,Tier3,Tier4,Tier5,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,NA,NA,Caixa e Equivalentes de Caixa,815.476,683.412
Ativo Total,Ativo Circulante,Contas a Receber,NA,NA,Contas a Receber,258.104,206.005
Ativo Total,Ativo Circulante,Contas a Receber,01,NA,Clientes,258.104,206.005
Ativo Total,Ativo Circulante,Estoques,NA,NA,Estoques,629.601,617.289
Ativo Total,Ativo Circulante,Tributos a Recuperar,NA,NA,Tributos a Recuperar,92.690,80.862
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,NA,NA,Outros Ativos Circulantes,79.343,64.382
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,03,NA,Outros,79.343,64.382
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,03,01,Ativos financeiros,12.153,16.059
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,03,03,Outras contas,53.490,39.511


<h4>Drop Columns:</h4>

In [158]:
data_frame_3 <- drop_cols(data_frame_3, round)
data_frame_3

Tier1,Tier2,Tier3,Tier4,Tier5,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,NA,NA,Caixa e Equivalentes de Caixa,815.476,683.412
Ativo Total,Ativo Circulante,Contas a Receber,01,NA,Clientes,258.104,206.005
Ativo Total,Ativo Circulante,Estoques,NA,NA,Estoques,629.601,617.289
Ativo Total,Ativo Circulante,Tributos a Recuperar,NA,NA,Tributos a Recuperar,92.690,80.862
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,03,NA,Outros,79.343,64.382
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,03,01,Ativos financeiros,12.153,16.059
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,03,03,Outras contas,53.490,39.511
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,03,04,Instrumentos financeiros derivativos,13.700,8.812
Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,09,NA,Créditos com Partes Relacionadas,12.773,9.730


#<h3>Fourth Round</h3>

In [170]:
round <- 4

<h4>Create Row Mask</h4>

In [171]:
row_mask <- create_row_mask(data_frame_3, round)
row_mask

Tier3,Tier4,Descrição da Conta
<chr>,<chr>,<chr>
Caixa e Equivalentes de Caixa,NA,Caixa e Equivalentes de Caixa
Contas a Receber,01,Clientes
Estoques,NA,Estoques
Tributos a Recuperar,NA,Tributos a Recuperar
Outros Ativos Circulantes,03,Outros
Ativo Realizável a Longo Prazo,09,Créditos com Partes Relacionadas
Ativo Realizável a Longo Prazo,10,Outros Ativos Não Circulantes
Investimentos,NA,Investimentos
Imobilizado,NA,Imobilizado


<h4>Left Join</h4>

In [172]:
data_frame_4 <- left_join(data_frame_3, round, row_mask)
data_frame_4

Tier1,Tier2,Tier3,Tier4,Tier5,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,NA,Caixa e Equivalentes de Caixa,815.476,683.412
Ativo Total,Ativo Circulante,Contas a Receber,Clientes,NA,Clientes,258.104,206.005
Ativo Total,Ativo Circulante,Estoques,Estoques,NA,Estoques,629.601,617.289
Ativo Total,Ativo Circulante,Tributos a Recuperar,Tributos a Recuperar,NA,Tributos a Recuperar,92.690,80.862
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros,NA,Outros,79.343,64.382
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros,01,Ativos financeiros,12.153,16.059
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros,03,Outras contas,53.490,39.511
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros,04,Instrumentos financeiros derivativos,13.700,8.812
Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Créditos com Partes Relacionadas,NA,Créditos com Partes Relacionadas,12.773,9.730


<h4>Drop Columns:</h4>

In [173]:
data_frame_4 <- drop_cols(data_frame_4, round)
data_frame_4

Tier1,Tier2,Tier3,Tier4,Tier5,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,NA,Caixa e Equivalentes de Caixa,815.476,683.412
Ativo Total,Ativo Circulante,Contas a Receber,Clientes,NA,Clientes,258.104,206.005
Ativo Total,Ativo Circulante,Estoques,Estoques,NA,Estoques,629.601,617.289
Ativo Total,Ativo Circulante,Tributos a Recuperar,Tributos a Recuperar,NA,Tributos a Recuperar,92.690,80.862
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros,01,Ativos financeiros,12.153,16.059
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros,03,Outras contas,53.490,39.511
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros,04,Instrumentos financeiros derivativos,13.700,8.812
Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Créditos com Partes Relacionadas,02,Créditos com Controladas,12.773,9.730
Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Outros Ativos Não Circulantes,03,Tributos a recuperar,90.374,80.006


#<h3>Fifth Round</h3>

In [175]:
data_frame_5 <- data_frame_4
data_frame_5[["Tier5"]] <- data_frame_5[["Descrição da Conta"]]
data_frame_5

Tier1,Tier2,Tier3,Tier4,Tier5,Descrição da Conta,Trimestre Atual,Exercício Anterior
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,Caixa e Equivalentes de Caixa,815.476,683.412
Ativo Total,Ativo Circulante,Contas a Receber,Clientes,Clientes,Clientes,258.104,206.005
Ativo Total,Ativo Circulante,Estoques,Estoques,Estoques,Estoques,629.601,617.289
Ativo Total,Ativo Circulante,Tributos a Recuperar,Tributos a Recuperar,Tributos a Recuperar,Tributos a Recuperar,92.690,80.862
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros,Ativos financeiros,Ativos financeiros,12.153,16.059
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros,Outras contas,Outras contas,53.490,39.511
Ativo Total,Ativo Circulante,Outros Ativos Circulantes,Outros,Instrumentos financeiros derivativos,Instrumentos financeiros derivativos,13.700,8.812
Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Créditos com Partes Relacionadas,Créditos com Controladas,Créditos com Controladas,12.773,9.730
Ativo Total,Ativo Não Circulante,Ativo Realizável a Longo Prazo,Outros Ativos Não Circulantes,Tributos a recuperar,Tributos a recuperar,90.374,80.006
